### Import Necessary Packages
1. Tensorflow for Machine Learning Functions
2. Numpy for numerical computation
3. os for dealing with directory
4. cv2 and plt for reading and plotting image
5. skimage for image resize

In [ ]:
import tensorflow as tf
import numpy as np
import os,random,cv2               
import matplotlib.pyplot as plt    
from skimage import transform

### To Supress the warning
By overriding warn function with manual empty function call

In [ ]:
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

### Reading Training and Testing Dataset
 Returns the list of filename present in given directory

In [ ]:
start = 0
def listFiles(path):
    dirs = os.listdir(path)
    filelists = []
    for i in dirs:
        filelists.append(os.path.join(path,i))        
    return filelists

def read_dataset(filelists,batch_size):
    imagedata = np.zeros((batch_size,49152),dtype=np.float32)                         
    labels = np.zeros((batch_size,2),dtype=np.float32)
    global start
    
    if (start+1)*batch_size > len(filelists):
        random.shuffle(filelists)
        start = 0
        
    for i in range(start*batch_size,(start+1)*batch_size):
        img = cv2.imread(filelists[i])        
        img = transform.resize(img,(128,128,3))
        img = img.reshape((1,49152))      
               
        st = filelists[i].find("cat")
        if(st != -1):
            label = [1,0]  # CAT
            labels[i-start*batch_size]=label
        else:
            label = [0,1]  # DOG
            labels[i-start*batch_size]=label
            
        imagedata[i-start*batch_size]=img       
        
    start = start+1   
    return imagedata.reshape((batch_size,49152)),labels.reshape((batch_size,2))
        

### Defining CONV and FC Layers
Convolution layer does feature extraction and fully connected layer does the classification

In [ ]:
def conv_layer(input,size_in,size_out,name="conv"):
    w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
    conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
    act = tf.nn.relu(conv + b)
    return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

def fc_layer(input,size_in,size_out,name="fc"):
    w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
    act = tf.matmul(input, w) + b  
    return act

### Creating a CNN Architecture
It describes how many layers are there in your model, how they are connected, what is kernel size, extent of kernel overlapping, types of activation and pooling  and type of optimizer and loss etc.

It consists 2 convolution and 1 fully connected layer

In [ ]:
def CNN():
    x = tf.placeholder(tf.float32, shape=[None, 49152], name="x")
    x_image = tf.reshape(x, [-1, 128, 128, 3])
    y = tf.placeholder(tf.float32, shape=[None, 2], name="labels")
    
    conv1 = conv_layer(x_image, 3, 32, "conv1")
    conv2 = conv_layer(conv1, 32, 64, "conv2")
    
    
    flatten = tf.reshape(conv2,[-1,32*32*64])    
    logits = fc_layer(flatten,32*32*64,2,"fc1")
    
    xent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y), name="xent")   
    train = tf.train.AdamOptimizer(0.001).minimize(xent)
    
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    
    filelists = listFiles("./train/")  # It consists nearly 22K images for training
    for i in range(250*10):            # 10 is epoch size
        if i%250==0:
            print(str(i)+" epoch is running")
        x_train,y_train = read_dataset(filelists,100)    # 100 is batch size    
        sess.run(train,feed_dict={x:x_train, y: y_train})
        
    

    global start    
    start = 0  
    filelists = listFiles("./test/")   # It contains nearly 3K images for testing
    random.shuffle(filelists)
    for i in range(12):  
        x_test,y_test = read_dataset(filelists,250)
        [a,b,c] = sess.run([train,accuracy,logits],feed_dict={x:x_test,y:y_test})
        print(b)
    
    

### Executing and Testing Accuracy
It means calling the function and checking how much accuracy we are getting. When I test it on 250 size batch, it gives me accuracy of around 75-80%. The mean goes around 78%.

In [ ]:
CNN()